In [1]:
# Load the gams extension
%load_ext gams_magic

# Job shop scheduling

A job consists of making three parts.  Part 1 has three operations that have to be
done in order. Part 2 has two operations that can be done in any order. Part 3
has three operations, both operation 1 and 2 must be done before operation 3.

There are two machines. Each operation has to be done on a particular machine.

### Minimize total completion time.  

Key issues to deal with are:
- A machine cannot process two tasks at once.
- Two operations on the same part cannot be processed concurrently.

Tasks are defined by pairs (part,operation) that have to get done!
Processing times (and their machines) are given below:

In [2]:
%%gams
set part /p1*p3/;
set oper /o1*o3/;
set machine /m1*m2/;

parameter procdata(part,oper,machine) /
p1.o1.m1      3
p1.o2.m1      5
p1.o3.m2      6
p2.o1.m1      8
p2.o2.m2      4
p3.o1.m2      9
p3.o2.m1      2
p3.o3.m2      7
/;

In [3]:
%%gams
# put gams code here
* create alias for part,oper, and machine
alias(part,part2);
alias(oper,oper2);
alias(machine,machine2);

parameter duration(part,oper);
* define duration using procdata that is provided by the problem
duration(part,oper) = sum(machine,procdata(part,oper,machine));

set task(part,oper);
task(part,oper) = yes$(duration(part,oper) gt 0);

set i(part,oper), j(part,oper);
i(task) = yes; j(task) = yes;

* define a set called precedence that will determine the order of operations happen on certain parts
set precedence(part,oper,part,oper)/
  p1.o1.p1.o2
  p1.o2.p1.o3
  p3.o1.p3.o3
  p3.o2.p3.o3
/;

* set disjoint parts
* Two operations on the same part cannot be processed concurrently.
* A machine cannot process two tasks at once.
set disjoint(part,oper,machine,part2,oper2,machine2);
disjoint(part,oper,machine,part2,oper2,machine2) = no;
disjoint(part,oper,machine,part2,oper2,machine)$(procdata(part,oper,machine) > 0 and procdata(part2,oper2,machine) > 0 and (ord(part) ne ord(part2) or ord(oper) ne ord(oper2))) = yes;
disjoint(part,oper,machine,part,oper2,machine2)$(ord(oper) ne ord(oper2)) = yes;

* create big M
scalar M;
M = sum((part,oper),duration(part,oper));

* create variables
free variable comp;
positive variables start(part,oper),comp_for_each_part(part);
binary variables order(part,oper,part,oper);

equations objective1 "this objective helps to calculate the total completion time",
          objective_final "calculate the total completion time",
          defstart "define start time for different tasks",
          disjoint1 "constraint for disjoint case",
          disjoint2 "constraint for disjoint case";
          
* helper for objective function
objective1(part,oper)..
  comp_for_each_part(part) =g= start(part,oper) + duration(part,oper);
  
* objective function 
objective_final..
  comp =e= sum(part,comp_for_each_part(part));

* equation definitions
defstart(i,j)$precedence(i,j)..
  start(i) + duration(i) =l= start(j);

disjoint1(i,machine,j,machine2)$disjoint(i,machine,j,machine2)..
  start(i) + procdata(i,machine) =l= start(j) + M*(1-order(i,j));

disjoint2(i,machine,j,machine2)$disjoint(i,machine,j,machine2)..
  start(j) + procdata(j,machine2) =l= start(i) + M*order(i,j);

model jobshop /all/;
jobshop.optcr = 0;
solve jobshop using mip min comp;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),58.0,166,47,MIP,CPLEX,0.038


In [4]:
# display start time of task, completion time of each part, total completion time
%gams_pull -d start comp_for_each_part comp
display(start,comp_for_each_part,comp)

,part,oper,level,marginal,lower,upper,scale
0,p1,o1,12.0,0.000000e+00,0.0,inf,1.0
1,p1,o2,15.0,0.000000e+00,0.0,inf,1.0
2,p1,o3,20.0,0.000000e+00,0.0,inf,1.0
3,p2,o1,4.0,0.000000e+00,0.0,inf,1.0
4,p2,o2,0.0,3.000000e+00,0.0,inf,1.0
5,p2,o3,0.0,4.940656e-324,0.0,inf,1.0
6,p3,o1,4.0,0.000000e+00,0.0,inf,1.0
7,p3,o2,0.0,4.940656e-324,0.0,inf,1.0
8,p3,o3,13.0,0.000000e+00,0.0,inf,1.0


,part,level,marginal,lower,upper,scale
0,p1,26.0,0.0,0.0,inf,1.0
1,p2,12.0,0.0,0.0,inf,1.0
2,p3,20.0,0.0,0.0,inf,1.0


,level,marginal,lower,upper,scale
0,58.0,0.0,-inf,inf,1.0
